In [1]:
import cobra
import os 
import sys
import pandas as pd
import numpy as np

In [2]:
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils
import ecpy

In [3]:
import importlib
importlib.reload(utils)
importlib.reload(ecpy)

<module 'ecpy' from '/Users/gangl/Documents/GitHub/Halo-GEM/ecpy/ecpy.py'>

#### 1. Load the GEM model

In [4]:
model_file = '../../../ModelFiles/json/Halo_GEM_v1.json'
model = cobra.io.load_json_model(model_file)

Using license file /Users/gangl/gurobi.lic
Academic license - for non-commercial use only


In [5]:
utils.report_model(model)

Reactions: 2269
Metabolites: 2339


In [ ]:
utils.set_bound()

In [6]:
utils.test_biomass_production(model)

cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


(<Solution infeasible at 0x13296872b0>, <Solution 1.342 at 0x132968e4a8>)

#### 2. Convert to irreversible model

In [38]:
irrModel = ecpy.convertToIrrev(model)

In [39]:
utils.report_model(irrModel)

Reactions: 2689
Metabolites: 2339


In [40]:
utils.test_biomass_production(irrModel)

cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


(<Solution infeasible at 0x1328596fd0>, <Solution 1.342 at 0x13284c5198>)

#### 3. Add enzymes into model: eModel, need kcat values

##### 3.1 prepare a dictionary with (enz_id,rxn_id) as keys and kcasts as values
In this reconstruction, kcats for reactions were matched only at the reaction level, meaning that all iso-enzymes that catalyze the same reaction have the same kcat.

In [41]:
df_enz_kcat = pd.read_csv('../Results/mapped_kcats.csv',index_col=0)
# In this file, all kcats are in the unit of 1/s and log10 transformed

In [42]:
df_enz_kcat.head(5)

,log10_kcat_mean,log10_kcat_std,log10_kcat_max,log10_kcat_max_std
RXN-6021,1.749244,0.915640,3.463043,0.285781
DEOXYRIBOSE-P-ALD-RXN,1.883541,0.734648,2.592954,0.319713
THREONINE-ALDOLASE-RXN,0.271842,1.000000,0.271842,1.000000
R02300_copy1,1.223489,0.118440,1.298665,0.052214
R02300_copy2,1.223489,0.118440,1.305291,0.034038


In [62]:
kcat_dict = ecpy.prepare_kcats_dict(irrModel,df_enz_kcat,'log10_kcat_mean')
# In this dictionary, all kcats are in the unit of 1/h 

In [63]:
len(kcat_dict)

2495

##### 3.2 add enzymes into reactions

In [64]:
eModel = ecpy.convertToEnzymeModel(irrModel,kcat_dict)

In [65]:
utils.report_model(eModel)

Reactions: 4678
Metabolites: 3832


In [66]:
utils.test_biomass_production(eModel)

cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


(<Solution infeasible at 0x132b0a9f98>, <Solution 1.342 at 0x132b0c4160>)

#### 4. add protein pool

In [67]:
df_enz_mw = pd.read_csv('../Results/enzyme_MWs.csv',index_col=0)
df_enz_mw.head(5)

,mw
TD01GL003599,35.419836
TD01GL000170,67.223844
TD01GL002075,33.944536
TD01GL002983,81.953455
TD01GL000394,20.907750


In [68]:
MWs = {ind:df_enz_mw.loc[ind,'mw'] for ind in df_enz_mw.index}

In [69]:
ecModel = ecpy.constrainPool(eModel,MWs,list(MWs.keys()),0.018)

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpc9w4vo1i.lp
Reading time = 0.04 seconds
: 3832 rows, 9356 columns, 36598 nonzeros


In [70]:
rxn_pool = ecModel.reactions.get_by_id('prot_pool_exchange')
print(rxn_pool.upper_bound)

0.018


In [72]:
rxn_pool.upper_bound = 1
utils.test_biomass_production(ecModel)

(<Solution 0.440 at 0x132b9866d8>, <Solution 0.906 at 0x132b7c0a20>)

In [57]:
ecModel.reactions.Exchange_Glucopyranose.upper_bound

inf